In [2]:
import pandas as pd 
df = pd.read_pickle('shared/Project-3_NYC_311_Calls.pkl')

In [3]:
df = df.set_index(pd.DatetimeIndex(df['Created Date']))
del df['Created Date']

In [5]:
df_2022 = df.loc['2022']

In [6]:
daily_complaints = df_2022['Unique Key'].resample('D').count()


In [8]:
#Q1

average_daily_complaints = daily_complaints.mean()
print(average_daily_complaints)

8684.320547945206


In [9]:
#Q2

daily_calls = df['Unique Key'].resample('D').count()

In [10]:
max_calls_date = daily_calls.idxmax()
print(max_calls_date)

2020-08-04 00:00:00


In [11]:
#Q3

df_max_date = df.loc['2020-08-04']
complaint_counts = df_max_date['Complaint Type'].value_counts()
print(complaint_counts)

Complaint Type
Damaged Tree                           14863
Noise - Residential                      982
Request Large Bulky Item Collection      909
Street Light Condition                   617
Overgrown Tree/Branches                  609
                                       ...  
Bus Stop Shelter Placement                 1
Unsanitary Pigeon Condition                1
Public Payphone Complaint                  1
For Hire Vehicle Report                    1
Bridge Condition                           1
Name: count, Length: 125, dtype: int64


In [12]:
#Q4

monthly_calls = df['Unique Key'].resample('M').count()

In [13]:
monthly_average_calls = monthly_calls.groupby(monthly_calls.index.month).mean()

In [14]:
quietest_month = monthly_average_calls.idxmin()
print(quietest_month)

2


In [22]:
#Q5

from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(daily_calls, model='additive')

In [23]:
seasonal_component = decomposition.seasonal

In [24]:
seasonal_value = round(seasonal_component['2020-12-25'])
print(seasonal_value)

183


In [25]:
#Q6

autocorrelation_lag_1 = daily_calls.autocorr(lag=1)
print(autocorrelation_lag_1)

0.7517059728398577


In [29]:
pip install prophet


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [38]:
from prophet import Prophet

In [39]:
#Q7

df_prophet = daily_calls.reset_index().rename(columns={'index': 'ds', 'Unique Key': 'y'})


In [40]:
train = df_prophet.iloc[:-90]
test = df_prophet.iloc[-90:]

In [45]:
train = train.rename(columns={'Created Date': 'ds'})

In [46]:
model = Prophet()
model.fit(train)

14:45:45 - cmdstanpy - INFO - Chain [1] start processing
14:45:46 - cmdstanpy - INFO - Chain [1] done processing


In [48]:
future = model.make_future_dataframe(periods=90)
forecast = model.predict(future)

In [49]:
from sklearn.metrics import mean_squared_error
from math import sqrt
rmse = sqrt(mean_squared_error(test['y'], forecast['yhat'].iloc[-90:]))
print(rmse)

1231.513760758433
